In [1]:
import os
os.environ['OPENAI_API_KEY'] = ''

from langchain.llms import OpenAI

In [2]:
llm = OpenAI(temperature = .9)

In [3]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Bright Feet Co.


## Chat models

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [9]:
chat = ChatOpenAI(temperature=0)

In [17]:
chat([HumanMessage(content="Translate this sentence from English to Korean. I love programming.")])

AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={})

- `generate`를 통해 다중의 메세지를 입력해서 문장을 생성할 수 있음
- 추가적인 메세지 파라미터와 함께 `LLMResult`를 return

In [19]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Korean."),
        HumanMessage(content="Translate this sentence from English to Korean. I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Korean."),
        HumanMessage(content="Translate this sentence from English to Korean. I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-cuWqlSy5Uo5bBRhRF5ks4d88 on requests per min. Limit: 3 / min. Please try again in 20s. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-cuWqlSy5Uo5bBRhRF5ks4d88 on requests per min. Limit: 3 / min. Please try again in 20s. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a pay

LLMResult(generations=[[ChatGeneration(text='저는 프로그래밍을 좋아합니다.', generation_info=None, message=AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={}))], [ChatGeneration(text='나는 인공지능을 사랑합니다.', generation_info=None, message=AIMessage(content='나는 인공지능을 사랑합니다.', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 73, 'completion_tokens': 26, 'total_tokens': 99}, 'model_name': 'gpt-3.5-turbo'})

In [20]:
result.llm_output

{'token_usage': {'prompt_tokens': 73,
  'completion_tokens': 26,
  'total_tokens': 99},
 'model_name': 'gpt-3.5-turbo'}

### PromptTemplates

In [21]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [22]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="Korean", text="I would like to programming").to_messages())

AIMessage(content='저는 프로그래밍을 하고 싶습니다.', additional_kwargs={})

### LLMChain

In [27]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [28]:
chain.run(input_language="English", output_language="Korean", text="I love programming.")

'저는 프로그래밍을 좋아합니다.'

## Moduels

### Indexes

`Text Splitters`, `embeddings`, `vectorstores`

1. 인덱스 생성
2. 인덱스로부터 `Retriever` 생성
3. QA Chain 생성
4. 질의하기

In [4]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

from langchain.document_loaders import TextLoader

In [5]:
from langchain.indexes import VectorstoreIndexCreator
loader = TextLoader('D:/lee.dr/Chatbot/data/state_of_the_union.txt', encoding = 'utf-8')

In [35]:
loader

In [6]:
from langchain.indexes import VectorstoreIndexCreator

In [7]:
index = VectorstoreIndexCreator().from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


In [8]:
query = 'What did the president say about Ketanji Brown Jackson'
index.query(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans."

In [9]:
query = "What did the president say about Ketanji Brown Jackson"
index.query_with_sources(query)

{'question': 'What did the president say about Ketanji Brown Jackson',
 'answer': " The president said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, one of the nation's top legal minds, to continue Justice Breyer's legacy of excellence. He also said that she has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.\n",
 'sources': 'D:/lee.dr/Chatbot/data/state_of_the_union.txt'}

### Walkthrough

1. Chunk단위로 Document 쪼개기기
2. 각 document를 위한 embedding 생성
3. `vectorstore`에 embedding과 document 저장장

In [10]:
documents = loader.load()

In [11]:
# split the documents into chunks
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(documents)

In [12]:
texts[0]

Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.', metadata={'source':

In [13]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [14]:
# index사용해 vectorstore 생성
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [15]:
retriever = db.as_retriever()

In [16]:
qa = RetrievalQA.from_chain_type(llm = OpenAI(), chain_type = 'stuff', retriever = retriever)

In [17]:
query = 'What did the president say about Ketanji Brown Jackson'
qa.run(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, from a family of public school educators and police officers, a consensus builder, and has received a broad range of support since being nominated."

In [18]:
query = 'Why was a NATO Alliance created?'
qa.run(query)

' The NATO Alliance was created to secure peace and stability in Europe after World War 2.'

In [19]:
query = 'Why was the NATO Alliance instituted?'
qa.run(query)

' The NATO Alliance was instituted to secure peace and stability in Europe after World War 2.'

In [21]:
query = '우크라니아 대통령이 유럽의회에 한 말은? say to korean'
qa.run(query)

' 젤렌스키 대통령이 유럽의회에 가르친 말은 "빛이 어둠을 이길 것입니다"입니다.'